# Find actions required for the benchmarks

In [2]:
import numpy as np
import grid2op
from lips.dataset.utils.powergrid_utils import get_kwargs_simulator_scenario, XDepthAgent
from lips.config import ConfigManager

In [3]:
config = ConfigManager()
env_kwargs = get_kwargs_simulator_scenario(config)
env = grid2op.make(**env_kwargs)
env.deactivate_forecast()

/home/milad/environments/lips/lib/python3.8/site-packages/lightsim2grid/gridmodel/_aux_add_trafo.py:33: UserWarning: There were some Nan in the pp_net.trafo["tap_neutral"], they have been replaced by 0
  warnings.warn("There were some Nan in the pp_net.trafo[\"tap_neutral\"], they have been replaced by 0")
/home/milad/environments/lips/lib/python3.8/site-packages/lightsim2grid/gridmodel/_aux_add_trafo.py:41: UserWarning: There were some Nan in the pp_net.trafo["tap_step_percent"], they have been replaced by 0
  warnings.warn("There were some Nan in the pp_net.trafo[\"tap_step_percent\"], they have been replaced by 0")
/home/milad/environments/lips/lib/python3.8/site-packages/lightsim2grid/gridmodel/_aux_add_trafo.py:46: UserWarning: There were some Nan in the pp_net.trafo["tap_pos"], they have been replaced by 0
  warnings.warn("There were some Nan in the pp_net.trafo[\"tap_pos\"], they have been replaced by 0")
/home/milad/environments/lips/lib/python3.8/site-packages/lightsim2grid/gr

## Instantiate `XDepthAgent`

In [4]:
agent = XDepthAgent(env.action_space)

### get all the action combinations per substation

In [5]:
all_actions = agent.get_action_list()

In [124]:
for sub_id in range(env.n_sub):
    sub_actions = all_actions[sub_id]
    if any(sub_actions):
        for action_id, action in enumerate(sub_actions):
            #print(action.get_topological_impact()[1])
            print(f"******* SUB: {sub_id}  ACTION: {action_id} **********")
            print(action)

******* SUB: 1  ACTION: 0 **********
This action will:
	 - NOT change anything to the injections
	 - NOT perform any redispatching action
	 - NOT modify any storage capacity
	 - NOT perform any curtailment
	 - NOT force any line status
	 - NOT switch any line status
	 - NOT switch anything in the topology
	 - Set the bus of the following element(s):
	 	 - Assign bus 1 to line (extremity) id 0 [on substation 1]
	 	 - Assign bus 1 to line (origin) id 2 [on substation 1]
	 	 - Assign bus 1 to line (origin) id 3 [on substation 1]
	 	 - Assign bus 1 to line (origin) id 4 [on substation 1]
	 	 - Assign bus 2 to generator id 0 [on substation 1]
	 	 - Assign bus 2 to load id 0 [on substation 1]
******* SUB: 1  ACTION: 1 **********
This action will:
	 - NOT change anything to the injections
	 - NOT perform any redispatching action
	 - NOT modify any storage capacity
	 - NOT perform any curtailment
	 - NOT force any line status
	 - NOT switch any line status
	 - NOT switch anything in the topolo

### Find equivalent actions that were considered for Benchmark2

In [31]:
REF_ACTION = [# sub_5_id1
              {"set_bus": {"loads_id": [(4, 2)], "generators_id": [(2, 2)], "lines_or_id": [(9, 2), (8, 2)]}},
              # sub_4_id1
              {"set_bus": {"loads_id": [(3, 2)], "lines_or_id": [(17, 2)], "lines_ex_id": [(6, 2)]}},
              {"set_bus": {"lines_or_id": [(17, 2)], "lines_ex_id": [(4, 2)]}},  # sub_4_id2
              {"set_bus": {"lines_or_id": [(15, 2)], "lines_ex_id": [(3, 2)]}},  # "sub3_id1"
              {"set_bus": {"lines_or_id": [(16, 2)], "lines_ex_id": [(3, 2)]}},  # "sub3_id2"
              {"set_bus": {"lines_or_id": [(6, 2), (16, 2)]}},  # "sub3_id3"
              {"set_bus": {"lines_or_id": [(15, 2)], "lines_ex_id": [(3, 2)], "loads_id": [(2, 2)]}},  # "sub3_id4"
              # "sub8_id1"
              {"set_bus": {"lines_or_id": [(10, 2), (19, 2)], "loads_id": [(5, 2)]}},
              # "sub8_id2"
              {"set_bus": {"lines_or_id": [(10, 2), (19, 2)]}},
              # "sub8_id3"
              {"set_bus": {"lines_or_id": [(10, 2)], "loads_id": [(5, 2)]}},
              {"set_bus": {"lines_or_id": [(4, 2), (2, 2)]}},  # "sub1_id1" \n",
              {"set_bus": {"lines_or_id": [(3, 2)], "generators_id": [(0, 2)]}},  # "sub1_id2"
              {"set_line_status": [(14, -1)]},  # "powerline_9_10"
              {"set_line_status": [(12, -1)]},  # "powerline_12_13"
              {"set_line_status": [(3, -1)]},  # "powerline_1_3"
              # composed action sub_5_id1 + sub_4_id1
              {"set_bus": {"loads_id": [(4, 2), (3, 2)],
                           "generators_id": [(2, 2)],
                           "lines_or_id": [(9, 2), (8, 2), (17, 2)],
                           "lines_ex_id": [(6, 2)]}},
              # composed action sub_5_id1 + sub8_id1
              {"set_bus": {"loads_id": [(4, 2), (5, 2)],
                           "generators_id": [(2, 2)],
                           "lines_or_id": [(9, 2), (8, 2), (10, 2), (19, 2)],
                           "lines_ex_id": []}},
              # composed action sub_4_id1 + sub8_id1
              {"set_bus": {"loads_id": [(3, 2), (5, 2)],
                           "lines_or_id": [(17, 2), (10, 2), (19, 2)],
                           "lines_ex_id": [(6, 2)]}},
              # composed action sub_1_id1 + sub_3_id4
              {"set_bus": {"loads_id": [(2, 2)],
                           "lines_or_id": [(15, 2), (4, 2), (2, 2)],
                           "lines_ex_id": [(3, 2)]}},
              ]

In [92]:
def _aux_act_scenario(env) -> tuple:
    """Auxiliary function to prepare required actions to take

    It returns singular topo actions (acting on one sub)

    Parameters
    ----------
    env : ``grid2op.Environment``
        the environment on which actions should be taken

    Returns
    -------
    ``tuple``
        list of topological action and list of N-1 actions
    """
    li_ref_topo = [{"set_line_status": [(l_id, -1)]} for l_id in range(env.n_line)]
    li_act_n1 = [REF_ACTION[0], REF_ACTION[1], REF_ACTION[6], REF_ACTION[7], REF_ACTION[10]]
    li_ref_topo = [env.action_space(el) for el in li_ref_topo]
    li_act_n1 = [env.action_space(el) for el in li_act_n1]
    return li_ref_topo, li_act_n1

def _aux_act_scenario_ood(env) -> tuple:
    """Auxiliary function to prepare required actions to take

    It returns composed topo actions (acting on two subs) to test the out-of-distribution

    Parameters
    ----------
    env : ``grid2op.Environment``
        the environment on which actions should be taken

    Returns
    -------
    ``tuple``
        list of topological action and list of N-1 actions
    """
    li_ref_topo = [{"set_line_status": [(l_id, -1)]} for l_id in range(env.n_line)]
    li_act_n1 = [REF_ACTION[15], REF_ACTION[16], REF_ACTION[17], REF_ACTION[18]]
    li_ref_topo = [env.action_space(el) for el in li_ref_topo]
    li_act_n1 = [env.action_space(el) for el in li_act_n1]
    return li_ref_topo, li_act_n1

def transform_action(action_space, action):
    sub_impacted = int(np.where(action.get_topological_impact()[1])[0])
    sub_to_one = env.action_space({"set_bus": {"substations_id":[(sub_impacted, np.ones(action_space.sub_info[sub_impacted], dtype=int))]}})
    return sub_to_one + action

def find_impacted_subs(action):
    sub_impacted = (np.where(action.get_topological_impact()[1])[0])
    return sub_impacted

In [126]:
li_ref_topo, li_act_n1 = _aux_act_scenario(env)
li_act_n1 = [transform_action(env.action_space, action) for action in li_act_n1]

In [79]:
for sub_id in range(env.n_sub):
    sub_actions = all_actions[sub_id]
    if any(sub_actions):
        for action_id, action in enumerate(sub_actions):
            #print(action.get_topological_impact()[1])
            if action in li_act_n1:
                print((sub_id, action_id))

(1, 13)
(4, 3)
(5, 42)


In [86]:
li_ref_topo, li_act_n1 = _aux_act_scenario_ood(env)
#li_act_n1 = [transform_action(env.action_space, action) for action in li_act_n1]

In [93]:
for action in li_act_n1:
    print(find_impacted_subs(action))

[4 5]
[5 8]
[4 8]
[1 3]


In [94]:
for sub_id in range(env.n_sub):
    sub_actions = all_actions[sub_id]
    if any(sub_actions):
        for action_id, action in enumerate(sub_actions):
            #print(action.get_topological_impact()[1])
            if action in li_act_n1:
                print((sub_id, action_id))

In [125]:
for action in li_act_n1:
    print(action)

This action will:
	 - NOT change anything to the injections
	 - NOT perform any redispatching action
	 - NOT modify any storage capacity
	 - NOT perform any curtailment
	 - NOT force any line status
	 - NOT switch any line status
	 - NOT switch anything in the topology
	 - Set the bus of the following element(s):
	 	 - Assign bus 1 to line (origin) id 7 [on substation 5]
	 	 - Assign bus 2 to line (origin) id 8 [on substation 5]
	 	 - Assign bus 2 to line (origin) id 9 [on substation 5]
	 	 - Assign bus 1 to line (extremity) id 17 [on substation 5]
	 	 - Assign bus 2 to generator id 2 [on substation 5]
	 	 - Assign bus 1 to generator id 3 [on substation 5]
	 	 - Assign bus 2 to load id 4 [on substation 5]
This action will:
	 - NOT change anything to the injections
	 - NOT perform any redispatching action
	 - NOT modify any storage capacity
	 - NOT perform any curtailment
	 - NOT force any line status
	 - NOT switch any line status
	 - NOT switch anything in the topology
	 - Set the bus

In [130]:
for el in li_ref_topo:
    print(el)

This action will:
	 - NOT change anything to the injections
	 - NOT perform any redispatching action
	 - NOT modify any storage capacity
	 - NOT perform any curtailment
	 - Force disconnection of 1 powerlines ([0])
	 - NOT switch any line status
	 - NOT switch anything in the topology
	 - NOT force any particular bus configuration
This action will:
	 - NOT change anything to the injections
	 - NOT perform any redispatching action
	 - NOT modify any storage capacity
	 - NOT perform any curtailment
	 - Force disconnection of 1 powerlines ([1])
	 - NOT switch any line status
	 - NOT switch anything in the topology
	 - NOT force any particular bus configuration
This action will:
	 - NOT change anything to the injections
	 - NOT perform any redispatching action
	 - NOT modify any storage capacity
	 - NOT perform any curtailment
	 - Force disconnection of 1 powerlines ([2])
	 - NOT switch any line status
	 - NOT switch anything in the topology
	 - NOT force any particular bus configuration
T

# Neurips Track 1 Dataset

In [120]:
import pathlib
LIPS_PATH = pathlib.Path().resolve().parent # it is supposed that the notebook had run from getting_started folder
CONFIG_PATH = LIPS_PATH / "lips" / "config" / "conf_nips_track1.ini"

In [121]:
config = ConfigManager(path=CONFIG_PATH)
env_kwargs = get_kwargs_simulator_scenario(config)
env_nips = grid2op.make(**env_kwargs)

/home/milad/environments/lips/lib/python3.8/site-packages/lightsim2grid/gridmodel/_aux_add_slack.py:113: UserWarning: We found either some slack coefficient to be < 0. or they were all 0.We set them all to 1.0 to avoid such issues
  warnings.warn("We found either some slack coefficient to be < 0. or they were all 0."


In [162]:
unit_sub_1 = {"set_bus": {"loads_id": [(2,2)], "lines_or_id": [(3, 2), (4, 2)]}} # cannot be found
unit_sub_1_bis = {"set_bus": {"loads_id": [(3,2)], "generators_id": [(0,2)], "lines_or_id": [(12, 2)]}}
action1 = env_nips.action_space(unit_sub_1)
action1_bis = env_nips.action_space(unit_sub_1_bis)
unit_sub_2 = {"set_bus": {"generators_id": [(0,2)], "lines_or_id": [(12, 2)]}}
action2 = env_nips.action_space(unit_sub_2)
unit_sub_16_1 = {"set_bus": {"lines_ex_id": [(17,2), (21,2)], "generators_id": [(6,2)]}}
unit_sub_16_1_bis = {"set_bus": {"lines_ex_id": [(18,2), (21,2)], "generators_id": [(6,2)]}}
action3 = env_nips.action_space(unit_sub_16_1)
action3_bis = env_nips.action_space(unit_sub_16_1_bis)
unit_sub_16_2 = {"set_bus": {"lines_ex_id": [(20,2)], "loads_id": [(17,2)]}}
action4 = env_nips.action_space(unit_sub_16_2)
required_actions = [action1, action1_bis, action2, action3, action3_bis, action4]
required_actions = [transform_action(env_nips.action_space, action) for action in required_actions]

In [169]:
int(np.where(action3._subs_impacted)[0])

16

In [135]:
agent = XDepthAgent(env.action_space)
all_actions = agent.all_topo_actions

In [163]:
for sub_id in range(env.n_sub):
    sub_actions = all_actions[sub_id]
    if any(sub_actions):
        for action_id, action in enumerate(sub_actions):
            #print(action.get_topological_impact()[1])
            if action in required_actions:
                print((sub_id, action_id))
                print(find_impacted_subs(action))
                print(action)

(1, 6)
[1]
This action will:
	 - NOT change anything to the injections
	 - NOT perform any redispatching action
	 - NOT modify any storage capacity
	 - NOT perform any curtailment
	 - NOT force any line status
	 - NOT switch any line status
	 - NOT switch anything in the topology
	 - Set the bus of the following element(s):
	 	 - Assign bus 1 to line (origin) id 3 [on substation 1]
	 	 - Assign bus 1 to line (origin) id 4 [on substation 1]
	 	 - Assign bus 2 to line (origin) id 12 [on substation 1]
	 	 - Assign bus 2 to generator id 0 [on substation 1]
	 	 - Assign bus 1 to load id 2 [on substation 1]
	 	 - Assign bus 1 to load id 3 [on substation 1]
(1, 7)
[1]
This action will:
	 - NOT change anything to the injections
	 - NOT perform any redispatching action
	 - NOT modify any storage capacity
	 - NOT perform any curtailment
	 - NOT force any line status
	 - NOT switch any line status
	 - NOT switch anything in the topology
	 - Set the bus of the following element(s):
	 	 - Assign bu

In [173]:
action = env.action_space({"set_line_status": [(9, -1)]})

In [175]:
obs, *_ = env.step(action)

In [198]:
my_list = []

In [199]:
my_list = [my_list.append(el) for el in np.where(~obs.line_status)]

In [200]:
my_list

[None]

In [229]:
obs.line_status[9] = True
obs.line_status[10] = True

In [230]:
disc_line = [int(el) for el in np.where(~obs.line_status)[0] if el.size > 0]

In [231]:
disc_line

[]

## New topo actions of nips track 1

In [19]:
topo_actions = [{"set_bus": {"substations_id": [[16, [1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2]]]}}, 
                {"set_bus": {"substations_id": [[28, [2, 1, 2, 1, 1]]]}}, 
                {"set_bus": {"substations_id": [[29, [2, 2, 1, 2, 1, 1, 2]]]}}, 
                {"set_bus": {"substations_id": [[29, [1, 2, 2, 2, 1, 2, 1]]]}}, 
                {"set_bus": {"substations_id": [[28, [2, 2, 2, 1, 1]]]}}, 
                {"set_bus": {"substations_id": [[29, [1, 2, 2, 1, 1, 1, 1]]]}}, {"set_bus": {"substations_id": [[29, [2, 1, 1, 1, 1, 2, 1]]]}}, {"set_bus": {"substations_id": [[32, [2, 2, 1, 1, 1]]]}}, {"set_bus": {"substations_id": [[28, [2, 1, 1, 2, 1]]]}}, {"set_bus": {"substations_id": [[26, [1, 1, 1, 1, 1, 2, 2, 1, 1]]]}}, {"set_bus": {"substations_id": [[21, [1, 2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[32, [1, 2, 2, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [2, 1, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[26, [1, 1, 1, 2, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[26, [2, 1, 2, 1, 1, 1, 1, 1, 1]]]}}, {"set_bus": {"substations_id": [[22, [2, 1, 2, 1, 1]]]}}, {"set_bus": {"substations_id": [[12, [2, 2, 1, 1]]]}}, {"set_bus": {"substations_id": [[12, [2, 1, 2, 1]]]}}, {"set_bus": {"substations_id": [[13, [1, 2, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[1, [1, 1, 2, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[4, [2, 1, 2, 1, 1, 1]]]}}, {"set_bus": {"substations_id": [[1, [1, 2, 2, 1, 2, 1]]]}}, {"set_bus": {"substations_id": [[4, [2, 2, 2, 1, 1, 1]]]}}, {"set_bus": {"substations_id": [[22, [1, 2, 2, 1, 1]]]}}, {"set_bus": {"substations_id": [[1, [2, 2, 1, 1, 2, 1]]]}}, {"set_bus": {"substations_id": [[13, [1, 2, 2, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [1, 1, 2, 2, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [1, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[4, [1, 2, 1, 1, 2, 2]]]}}, {"set_bus": {"substations_id": [[4, [2, 2, 2, 1, 2, 1]]]}}, {"set_bus": {"substations_id": [[16, [2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[13, [2, 2, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[26, [2, 2, 2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[26, [2, 2, 2, 2, 1, 1, 2, 1, 2]]]}}, {"set_bus": {"substations_id": [[12, [1, 2, 2, 1]]]}}, {"set_bus": {"substations_id": [[22, [1, 1, 1, 1, 1]]]}}, {"set_bus": {"substations_id": [[26, [2, 1, 1, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[22, [1, 2, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[21, [1, 1, 2, 2, 2, 1, 1, 1]]]}}, {"set_bus": {"substations_id": [[16, [1, 2, 2, 1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [2, 1, 2, 1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[33, [2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[23, [2, 2, 1, 1, 2, 2, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[22, [2, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[4, [1, 2, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[29, [2, 2, 2, 1, 1, 1, 1]]]}}, {"set_bus": {"substations_id": [[27, [2, 2, 1, 1, 1]]]}}, {"set_bus": {"substations_id": [[29, [1, 1, 2, 2, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[23, [2, 2, 1, 1, 1, 1, 1, 1, 2, 2]]]}}, {"set_bus": {"substations_id": [[21, [1, 2, 2, 2, 2, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[1, [1, 1, 2, 1, 2, 1]]]}}, {"set_bus": {"substations_id": [[31, [1, 1, 1]]]}}, {"set_bus": {"substations_id": [[16, [2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [1, 1, 2, 1, 1, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [1, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[32, [2, 1, 2, 1, 2]]]}}, {"set_bus": {"substations_id": [[9, [1, 1, 2, 2, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [2, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [2, 1, 2, 2, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[22, [2, 2, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[32, [1, 2, 2, 1, 1]]]}}, {"set_bus": {"substations_id": [[26, [2, 1, 1, 2, 1, 1, 1, 1, 1]]]}}, {"set_bus": {"substations_id": [[16, [1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[26, [1, 1, 2, 1, 1, 2, 2, 1, 1]]]}}, {"set_bus": {"substations_id": [[26, [1, 1, 2, 1, 1, 2, 1, 1, 1]]]}}, {"set_bus": {"substations_id": [[4, [1, 2, 2, 1, 2, 2]]]}}, {"set_bus": {"substations_id": [[1, [1, 2, 2, 1, 1, 1]]]}}, {"set_bus": {"substations_id": [[16, [1, 1, 1, 2, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[4, [2, 2, 1, 1, 1, 1]]]}}, {"set_bus": {"substations_id": [[9, [2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[26, [1, 1, 2, 1, 2, 2, 2, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[13, [2, 1, 2, 1, 1]]]}}, {"set_bus": {"substations_id": [[26, [1, 1, 2, 1, 1, 1, 2, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [1, 1, 2, 1, 2, 2, 2, 2, 1, 1, 2, 1, 1, 2, 2, 1, 2]]]}}, {"set_bus": {"substations_id": [[23, [2, 2, 2, 1, 2, 1, 1, 2, 1, 2]]]}}, {"set_bus": {"substations_id": [[32, [1, 2, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[1, [1, 2, 1, 1, 2, 1]]]}}, {"set_bus": {"substations_id": [[23, [2, 2, 2, 2, 1, 2, 1, 1, 2, 2]]]}}, {"set_bus": {"substations_id": [[21, [2, 2, 1, 2, 2, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[22, [1, 1, 2, 1, 2]]]}}, {"set_bus": {"substations_id": [[26, [1, 1, 2, 2, 2, 2, 2, 1, 1]]]}}, {"set_bus": {"substations_id": [[16, [2, 1, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [2, 1, 2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [2, 1, 2, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[28, [1, 1, 2, 2, 1]]]}}, {"set_bus": {"substations_id": [[7, [2, 2, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[26, [2, 2, 2, 2, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[26, [2, 2, 1, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[33, [2, 1, 2, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[28, [1, 2, 2, 1, 1]]]}}, {"set_bus": {"substations_id": [[26, [1, 1, 2, 1, 2, 1, 1, 1, 1]]]}}, {"set_bus": {"substations_id": [[27, [2, 2, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[26, [2, 2, 1, 2, 2, 1, 2, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[21, [2, 2, 2, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[18, [1, 2, 2, 1]]]}}, {"set_bus": {"substations_id": [[23, [2, 1, 2, 2, 2, 2, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[21, [2, 2, 1, 1, 1, 2, 1, 2]]]}}, {"set_bus": {"substations_id": [[21, [2, 2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[23, [2, 2, 2, 2, 2, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[21, [1, 1, 2, 2, 1, 1, 1, 1]]]}}, {"set_bus": {"substations_id": [[16, [2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[9, [2, 2, 1, 1, 1, 2, 1]]]}}, {"set_bus": {"substations_id": [[23, [2, 2, 1, 1, 1, 2, 1, 1, 2, 2]]]}}, {"set_bus": {"substations_id": [[23, [1, 1, 2, 2, 2, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[28, [1, 2, 1, 2, 1]]]}}, {"set_bus": {"substations_id": [[23, [1, 1, 1, 2, 2, 2, 1, 1, 2, 2]]]}}, {"set_bus": {"substations_id": [[16, [1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[33, [2, 2, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[33, [2, 1, 2, 2, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [2, 1, 1, 1, 2, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [2, 1, 2, 2, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[28, [2, 2, 1, 1, 1]]]}}, {"set_bus": {"substations_id": [[26, [1, 1, 2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [2, 1, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[1, [2, 1, 1, 1, 2, 2]]]}}, {"set_bus": {"substations_id": [[23, [2, 2, 2, 2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[21, [2, 2, 2, 1, 2, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[23, [2, 2, 1, 1, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[26, [2, 1, 2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[23, [1, 2, 1, 1, 2, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [2, 1, 2, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [2, 1, 1, 1, 1, 2, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[13, [2, 1, 2, 1, 2]]]}}, {"set_bus": {"substations_id": [[26, [1, 1, 2, 1, 1, 2, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[26, [2, 1, 2, 2, 2, 2, 2, 1, 2]]]}}, {"set_bus": {"substations_id": [[1, [2, 2, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [1, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [1, 1, 2, 1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[4, [1, 1, 2, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [2, 1, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [1, 2, 1, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[26, [1, 1, 1, 1, 1, 2, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [1, 1, 2, 2, 1, 2, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[32, [2, 2, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[26, [2, 1, 1, 2, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[26, [1, 1, 1, 2, 2, 2, 2, 1, 1]]]}}, {"set_bus": {"substations_id": [[26, [1, 1, 2, 2, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[23, [2, 2, 1, 2, 2, 2, 1, 1, 2, 2]]]}}, {"set_bus": {"substations_id": [[16, [2, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [1, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[16, [1, 1, 1, 2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2]]]}}, {"set_bus": {"substations_id": [[23, [2, 1, 1, 1, 1, 1, 1, 1, 1, 2]]]}}]

In [113]:
new_actions = []
grid2op_actions = []
for action in topo_actions:
    sub_id = action["set_bus"]["substations_id"][0][0]
    topo_vect = action["set_bus"]["substations_id"][0][1]
    new_actions.append({"set_bus": {"substations_id": [(sub_id, topo_vect)]}})
    grid2op_actions.append(env_nips.action_space(new_actions[-1]))

In [115]:
for action in grid2op_actions:
    print(action)

This action will:
	 - NOT change anything to the injections
	 - NOT perform any redispatching action
	 - NOT modify any storage capacity
	 - NOT perform any curtailment
	 - NOT force any line status
	 - NOT switch any line status
	 - NOT switch anything in the topology
	 - Set the bus of the following element(s):
	 	 - Assign bus 1 to line (extremity) id 17 [on substation 16]
	 	 - Assign bus 1 to line (extremity) id 18 [on substation 16]
	 	 - Assign bus 1 to line (extremity) id 19 [on substation 16]
	 	 - Assign bus 2 to line (extremity) id 20 [on substation 16]
	 	 - Assign bus 2 to line (extremity) id 21 [on substation 16]
	 	 - Assign bus 1 to line (origin) id 22 [on substation 16]
	 	 - Assign bus 1 to line (origin) id 23 [on substation 16]
	 	 - Assign bus 1 to line (origin) id 27 [on substation 16]
	 	 - Assign bus 2 to line (origin) id 28 [on substation 16]
	 	 - Assign bus 1 to line (origin) id 48 [on substation 16]
	 	 - Assign bus 1 to line (origin) id 49 [on substation 16]

This action will:
	 - NOT change anything to the injections
	 - NOT perform any redispatching action
	 - NOT modify any storage capacity
	 - NOT perform any curtailment
	 - NOT force any line status
	 - NOT switch any line status
	 - NOT switch anything in the topology
	 - Set the bus of the following element(s):
	 	 - Assign bus 2 to line (origin) id 3 [on substation 1]
	 	 - Assign bus 1 to line (origin) id 4 [on substation 1]
	 	 - Assign bus 1 to line (origin) id 12 [on substation 1]
	 	 - Assign bus 1 to generator id 0 [on substation 1]
	 	 - Assign bus 2 to load id 2 [on substation 1]
	 	 - Assign bus 2 to load id 3 [on substation 1]
This action will:
	 - NOT change anything to the injections
	 - NOT perform any redispatching action
	 - NOT modify any storage capacity
	 - NOT perform any curtailment
	 - NOT force any line status
	 - NOT switch any line status
	 - NOT switch anything in the topology
	 - Set the bus of the following element(s):
	 	 - Assign bus 2 to line (extremity

In [117]:
print(new_actions)

[{'set_bus': {'substations_id': [(16, [1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(28, [2, 1, 2, 1, 1])]}}, {'set_bus': {'substations_id': [(29, [2, 2, 1, 2, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(29, [1, 2, 2, 2, 1, 2, 1])]}}, {'set_bus': {'substations_id': [(28, [2, 2, 2, 1, 1])]}}, {'set_bus': {'substations_id': [(29, [1, 2, 2, 1, 1, 1, 1])]}}, {'set_bus': {'substations_id': [(29, [2, 1, 1, 1, 1, 2, 1])]}}, {'set_bus': {'substations_id': [(32, [2, 2, 1, 1, 1])]}}, {'set_bus': {'substations_id': [(28, [2, 1, 1, 2, 1])]}}, {'set_bus': {'substations_id': [(26, [1, 1, 1, 1, 1, 2, 2, 1, 1])]}}, {'set_bus': {'substations_id': [(21, [1, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(32, [1, 2, 2, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [2, 1, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(2

In [118]:
topo_actions = [{'set_bus': {'substations_id': [(16, [1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(28, [2, 1, 2, 1, 1])]}}, {'set_bus': {'substations_id': [(29, [2, 2, 1, 2, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(29, [1, 2, 2, 2, 1, 2, 1])]}}, {'set_bus': {'substations_id': [(28, [2, 2, 2, 1, 1])]}}, {'set_bus': {'substations_id': [(29, [1, 2, 2, 1, 1, 1, 1])]}}, {'set_bus': {'substations_id': [(29, [2, 1, 1, 1, 1, 2, 1])]}}, {'set_bus': {'substations_id': [(32, [2, 2, 1, 1, 1])]}}, {'set_bus': {'substations_id': [(28, [2, 1, 1, 2, 1])]}}, {'set_bus': {'substations_id': [(26, [1, 1, 1, 1, 1, 2, 2, 1, 1])]}}, {'set_bus': {'substations_id': [(21, [1, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(32, [1, 2, 2, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [2, 1, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(26, [1, 1, 1, 2, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(26, [2, 1, 2, 1, 1, 1, 1, 1, 1])]}}, {'set_bus': {'substations_id': [(22, [2, 1, 2, 1, 1])]}}, {'set_bus': {'substations_id': [(12, [2, 2, 1, 1])]}}, {'set_bus': {'substations_id': [(12, [2, 1, 2, 1])]}}, {'set_bus': {'substations_id': [(13, [1, 2, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(1, [1, 1, 2, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(4, [2, 1, 2, 1, 1, 1])]}}, {'set_bus': {'substations_id': [(1, [1, 2, 2, 1, 2, 1])]}}, {'set_bus': {'substations_id': [(4, [2, 2, 2, 1, 1, 1])]}}, {'set_bus': {'substations_id': [(22, [1, 2, 2, 1, 1])]}}, {'set_bus': {'substations_id': [(1, [2, 2, 1, 1, 2, 1])]}}, {'set_bus': {'substations_id': [(13, [1, 2, 2, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [1, 1, 2, 2, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [1, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(4, [1, 2, 1, 1, 2, 2])]}}, {'set_bus': {'substations_id': [(4, [2, 2, 2, 1, 2, 1])]}}, {'set_bus': {'substations_id': [(16, [2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(13, [2, 2, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(26, [2, 2, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(26, [2, 2, 2, 2, 1, 1, 2, 1, 2])]}}, {'set_bus': {'substations_id': [(12, [1, 2, 2, 1])]}}, {'set_bus': {'substations_id': [(22, [1, 1, 1, 1, 1])]}}, {'set_bus': {'substations_id': [(26, [2, 1, 1, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(22, [1, 2, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(21, [1, 1, 2, 2, 2, 1, 1, 1])]}}, {'set_bus': {'substations_id': [(16, [1, 2, 2, 1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [2, 1, 2, 1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(33, [2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(23, [2, 2, 1, 1, 2, 2, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(22, [2, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(4, [1, 2, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(29, [2, 2, 2, 1, 1, 1, 1])]}}, {'set_bus': {'substations_id': [(27, [2, 2, 1, 1, 1])]}}, {'set_bus': {'substations_id': [(29, [1, 1, 2, 2, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(23, [2, 2, 1, 1, 1, 1, 1, 1, 2, 2])]}}, {'set_bus': {'substations_id': [(21, [1, 2, 2, 2, 2, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(1, [1, 1, 2, 1, 2, 1])]}}, {'set_bus': {'substations_id': [(31, [1, 1, 1])]}}, {'set_bus': {'substations_id': [(16, [2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [1, 1, 2, 1, 1, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [1, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(32, [2, 1, 2, 1, 2])]}}, {'set_bus': {'substations_id': [(9, [1, 1, 2, 2, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [2, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [2, 1, 2, 2, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(22, [2, 2, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(32, [1, 2, 2, 1, 1])]}}, {'set_bus': {'substations_id': [(26, [2, 1, 1, 2, 1, 1, 1, 1, 1])]}}, {'set_bus': {'substations_id': [(16, [1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(26, [1, 1, 2, 1, 1, 2, 2, 1, 1])]}}, {'set_bus': {'substations_id': [(26, [1, 1, 2, 1, 1, 2, 1, 1, 1])]}}, {'set_bus': {'substations_id': [(4, [1, 2, 2, 1, 2, 2])]}}, {'set_bus': {'substations_id': [(1, [1, 2, 2, 1, 1, 1])]}}, {'set_bus': {'substations_id': [(16, [1, 1, 1, 2, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(4, [2, 2, 1, 1, 1, 1])]}}, {'set_bus': {'substations_id': [(9, [2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(26, [1, 1, 2, 1, 2, 2, 2, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(13, [2, 1, 2, 1, 1])]}}, {'set_bus': {'substations_id': [(26, [1, 1, 2, 1, 1, 1, 2, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [1, 1, 2, 1, 2, 2, 2, 2, 1, 1, 2, 1, 1, 2, 2, 1, 2])]}}, {'set_bus': {'substations_id': [(23, [2, 2, 2, 1, 2, 1, 1, 2, 1, 2])]}}, {'set_bus': {'substations_id': [(32, [1, 2, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(1, [1, 2, 1, 1, 2, 1])]}}, {'set_bus': {'substations_id': [(23, [2, 2, 2, 2, 1, 2, 1, 1, 2, 2])]}}, {'set_bus': {'substations_id': [(21, [2, 2, 1, 2, 2, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(22, [1, 1, 2, 1, 2])]}}, {'set_bus': {'substations_id': [(26, [1, 1, 2, 2, 2, 2, 2, 1, 1])]}}, {'set_bus': {'substations_id': [(16, [2, 1, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [2, 1, 2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [2, 1, 2, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(28, [1, 1, 2, 2, 1])]}}, {'set_bus': {'substations_id': [(7, [2, 2, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(26, [2, 2, 2, 2, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(26, [2, 2, 1, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(33, [2, 1, 2, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(28, [1, 2, 2, 1, 1])]}}, {'set_bus': {'substations_id': [(26, [1, 1, 2, 1, 2, 1, 1, 1, 1])]}}, {'set_bus': {'substations_id': [(27, [2, 2, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(26, [2, 2, 1, 2, 2, 1, 2, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(21, [2, 2, 2, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(18, [1, 2, 2, 1])]}}, {'set_bus': {'substations_id': [(23, [2, 1, 2, 2, 2, 2, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(21, [2, 2, 1, 1, 1, 2, 1, 2])]}}, {'set_bus': {'substations_id': [(21, [2, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(23, [2, 2, 2, 2, 2, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(21, [1, 1, 2, 2, 1, 1, 1, 1])]}}, {'set_bus': {'substations_id': [(16, [2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(9, [2, 2, 1, 1, 1, 2, 1])]}}, {'set_bus': {'substations_id': [(23, [2, 2, 1, 1, 1, 2, 1, 1, 2, 2])]}}, {'set_bus': {'substations_id': [(23, [1, 1, 2, 2, 2, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(28, [1, 2, 1, 2, 1])]}}, {'set_bus': {'substations_id': [(23, [1, 1, 1, 2, 2, 2, 1, 1, 2, 2])]}}, {'set_bus': {'substations_id': [(16, [1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(33, [2, 2, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(33, [2, 1, 2, 2, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [2, 1, 1, 1, 2, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [2, 1, 2, 2, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(28, [2, 2, 1, 1, 1])]}}, {'set_bus': {'substations_id': [(26, [1, 1, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [2, 1, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(1, [2, 1, 1, 1, 2, 2])]}}, {'set_bus': {'substations_id': [(23, [2, 2, 2, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(21, [2, 2, 2, 1, 2, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(23, [2, 2, 1, 1, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(26, [2, 1, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(23, [1, 2, 1, 1, 2, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [2, 1, 2, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [2, 1, 1, 1, 1, 2, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(13, [2, 1, 2, 1, 2])]}}, {'set_bus': {'substations_id': [(26, [1, 1, 2, 1, 1, 2, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(26, [2, 1, 2, 2, 2, 2, 2, 1, 2])]}}, {'set_bus': {'substations_id': [(1, [2, 2, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [1, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [1, 1, 2, 1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(4, [1, 1, 2, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [2, 1, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [1, 2, 1, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(26, [1, 1, 1, 1, 1, 2, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [1, 1, 2, 2, 1, 2, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(32, [2, 2, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(26, [2, 1, 1, 2, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(26, [1, 1, 1, 2, 2, 2, 2, 1, 1])]}}, {'set_bus': {'substations_id': [(26, [1, 1, 2, 2, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(23, [2, 2, 1, 2, 2, 2, 1, 1, 2, 2])]}}, {'set_bus': {'substations_id': [(16, [2, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [1, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(16, [1, 1, 1, 2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2])]}}, {'set_bus': {'substations_id': [(23, [2, 1, 1, 1, 1, 1, 1, 1, 1, 2])]}}]

In [119]:
topo_actions

[{'set_bus': {'substations_id': [(16,
     [1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2])]}},
 {'set_bus': {'substations_id': [(28, [2, 1, 2, 1, 1])]}},
 {'set_bus': {'substations_id': [(29, [2, 2, 1, 2, 1, 1, 2])]}},
 {'set_bus': {'substations_id': [(29, [1, 2, 2, 2, 1, 2, 1])]}},
 {'set_bus': {'substations_id': [(28, [2, 2, 2, 1, 1])]}},
 {'set_bus': {'substations_id': [(29, [1, 2, 2, 1, 1, 1, 1])]}},
 {'set_bus': {'substations_id': [(29, [2, 1, 1, 1, 1, 2, 1])]}},
 {'set_bus': {'substations_id': [(32, [2, 2, 1, 1, 1])]}},
 {'set_bus': {'substations_id': [(28, [2, 1, 1, 2, 1])]}},
 {'set_bus': {'substations_id': [(26, [1, 1, 1, 1, 1, 2, 2, 1, 1])]}},
 {'set_bus': {'substations_id': [(21, [1, 2, 1, 1, 1, 1, 1, 2])]}},
 {'set_bus': {'substations_id': [(32, [1, 2, 2, 1, 2])]}},
 {'set_bus': {'substations_id': [(16,
     [1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2])]}},
 {'set_bus': {'substations_id': [(16,
     [2, 1, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2])]}},
 {'set

In [136]:
config = ConfigManager(section_name="Benchmark2", path=CONFIG_PATH)

In [137]:
config.get_options_dict()

{'attr_x': ('prod_p', 'prod_v', 'load_p', 'load_q'),
 'attr_tau': ('line_status', 'topo_vect'),
 'attr_y': ('a_or',
  'a_ex',
  'p_or',
  'p_ex',
  'q_or',
  'q_ex',
  'prod_q',
  'load_v',
  'v_or',
  'v_ex'),
 'dataset_create_params': {'reference_args': {'prob_depth': (0.5, 0.5),
   'prob_type': (0.0, 1.0),
   'prob_do_nothing': 0.2,
   'max_disc': 2},
  'train': {'topo_actions': [{'set_bus': {'substations_id': [(16,
        [1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2])]}},
    {'set_bus': {'substations_id': [(28, [2, 1, 2, 1, 1])]}},
    {'set_bus': {'substations_id': [(29, [2, 2, 1, 2, 1, 1, 2])]}},
    {'set_bus': {'substations_id': [(29, [1, 2, 2, 2, 1, 2, 1])]}},
    {'set_bus': {'substations_id': [(28, [2, 2, 2, 1, 1])]}},
    {'set_bus': {'substations_id': [(29, [1, 2, 2, 1, 1, 1, 1])]}},
    {'set_bus': {'substations_id': [(29, [2, 1, 1, 1, 1, 2, 1])]}},
    {'set_bus': {'substations_id': [(32, [2, 2, 1, 1, 1])]}},
    {'set_bus': {'substations_id': [(28, [2, 1, 1, 2, 